In [1]:
#PROBLEM - A: WHEN SHOULD I MAKE A TRIP TO LONDON - WHEN ARE RENTAL PRICES GENERALLY LOW?
#PROBLEM - B: WHERE SHOULD I STAY - WHICH LOCATION HAS GOOD RATINGS AND GREAT HOSTS?

# STAGE 1 - STUDY THE DATASETS 
# STAGE 2 - FIND PATTERNS ACROSS TIME 
# STAGE 3 - FIND PATTERNS ACROSS SPACE - GEOGRAPHY 
# STAGE 4 - USE LINEAR REGRESSION TO PREDICT PRICE
# STAGE 5 - USE KMEANS CLUSTER TO SEGMENT HOSTS 

In [ ]:
### STAGE 1 - STUDY THE DATASETS ###
# Datasource - http://insideairbnb.com/get-the-data.html > London > Listings.csv; last scraped on 11 June 2020. 

In [73]:
import pandas as pd
import numpy as np

In [100]:
csv_list = ['listings-InsideAirbnb-11062020.csv',
           'listings-detailed-InsideAirbnb-11062020.csv',
           'calendar-insideairbnb-11062020.csv']

def opener(list_):
    n = 0
    dfs = []
    while n < len(list_):
        df = pd.read_csv(list_[n])
        print("-----------DF no.", n+1, "---------------")
        print(df.shape)
        print(df.head())
        dfs.append(df)
        n += 1
    return dfs

In [101]:
dfs_list = opener(csv_list)

-----------DF no. 1 ---------------
(83711, 16)
      id                                               name  host_id  \
0  11551         Arty and Bright London Apartment in Zone 2    43039   
1  13913                Holiday London DB Room Let-on going    54730   
2  15400                Bright Chelsea  Apartment. Chelsea!    60302   
3  17402              Superb 3-Bed/2 Bath & Wifi: Trendy W1    67564   
4  17506  Boutique Chelsea/Fulham Double bed 5-star ensuite    67915   

   host_name  neighbourhood_group           neighbourhood  latitude  \
0    Adriano                  NaN                 Lambeth  51.46225   
1      Alina                  NaN               Islington  51.56802   
2   Philippa                  NaN  Kensington and Chelsea  51.48796   
3        Liz                  NaN             Westminster  51.52195   
4  Charlotte                  NaN  Hammersmith and Fulham  51.47935   

   longitude        room_type  price  minimum_nights  number_of_reviews  \
0   -0.11732  Ent

In [102]:
#Inspect the columns of each dataframe
columns = dfs_list[2].columns

for i in columns:
    print (i)

listing_id
date
available
price
adjusted_price
minimum_nights
maximum_nights


In [103]:
date_price_df = dfs_list[2]
# CHECKPOINT
date_price_df.to_csv('date_price.csv')

In [122]:
# CHECKPOINT - START HERE #
date_price_df = pd.read_csv('date_price.csv')
date_price_df.head()

,Unnamed: 0,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,0,62901,2020-06-17,f,$55.00,$55.00,1.0,730.0
1,1,147244,2020-06-15,f,$149.00,$149.00,3.0,90.0
2,2,147244,2020-06-16,f,$149.00,$149.00,3.0,90.0
3,3,147244,2020-06-17,f,$149.00,$149.00,3.0,90.0
4,4,147244,2020-06-18,f,$149.00,$149.00,3.0,90.0


In [123]:
#useful columns - date & prices in list 1
df = date_price_df.loc[:, ["listing_id","date", "price"]]
# price_df.to_csv('price.csv')
print(df.shape)
df.head()
print("No. of unique listings is", df['listing_id'].nunique())
print("No. of unique dates is", df['date'].nunique())
##observation - each location may have multiple listings 

(30553644, 3)
No. of unique listings is 83711
No. of unique dates is 375


In [124]:
df.isnull().sum()
df.dropna(inplace = True)

In [125]:
print(df.shape)
print(df.isnull().sum())

(30552882, 3)
listing_id    0
date          0
price         0
dtype: int64


In [126]:
### Transform columns into the right format
##change date from str to datetime
df['dt'] = pd.to_datetime(df['date'])
df.head()

,listing_id,date,price,dt
0,62901,2020-06-17,$55.00,2020-06-17
1,147244,2020-06-15,$149.00,2020-06-15
2,147244,2020-06-16,$149.00,2020-06-16
3,147244,2020-06-17,$149.00,2020-06-17
4,147244,2020-06-18,$149.00,2020-06-18


In [127]:
##change price from str to float
def currency_converter(n):
    new_n = n.replace('$', '').replace(',', '')
    return new_n

In [128]:
df['price_num'] = df['price'].apply(currency_converter)
df.head()

,listing_id,date,price,dt,price_num
0,62901,2020-06-17,$55.00,2020-06-17,55.00
1,147244,2020-06-15,$149.00,2020-06-15,149.00
2,147244,2020-06-16,$149.00,2020-06-16,149.00
3,147244,2020-06-17,$149.00,2020-06-17,149.00
4,147244,2020-06-18,$149.00,2020-06-18,149.00


In [ ]:
#Explore the following:
## check number of unique listing ids; if there are multiple listings from the same id, then groupby listing id
#average out the px and check price distribution? 
## step 1 - check price distribution by count
## step 2 - check price distriution by date

time_gb = df.groupby(df['dt'].dt.strftime('%B')).sum()
time_gb

Unnamed: 0       0
listing_id       0
date             0
price         1118
price_num     1118
dtype: int64

In [44]:
#Drop rows without pricing data


Unnamed: 0    0
listing_id    0
date          0
price         0
price_num     0
dtype: int64

In [46]:
price_df['price_num'] = price_df['price'].apply(currency_converter)
price_df.head()

,Unnamed: 0,listing_id,date,price,price_num
0,0,171686,2019-09-15,$90.00,90.00
1,1,81449,2019-09-15,$141.00,141.00
2,2,81449,2019-09-16,$141.00,141.00
3,3,81449,2019-09-17,$141.00,141.00
4,4,81449,2019-09-18,$141.00,141.00


In [54]:
price_df['dt'] = pd.to_datetime(price_df['date'])
price_df.head()

Unnamed: 0             int64
listing_id             int64
date                  object
price                 object
price_num             object
dt            datetime64[ns]
dtype: object

In [55]:
price_df.dtypes

Unnamed: 0             int64
listing_id             int64
date                  object
price                 object
price_num             object
dt            datetime64[ns]
dtype: object

In [52]:
price_id_gb = price_df.groupby(['listing_id'])
mean_price = price_id_gb['price_num'].mean
mean_price
#the list represents daily rates of 85273 hosts

<bound method GroupBy.mean of <pandas.core.groupby.generic.SeriesGroupBy object at 0x0000025FD20A0550>>

In [59]:
price_dt_gb = price_df.groupby(['dt'])
seasonal_price = price_dt_gb['price_num'].describe()
seasonal_price

,count,unique,top,freq
dt,,,,
2019-09-14,18878,540,100.00,729
2019-09-15,85265,909,50.00,2778
2019-09-16,85265,907,50.00,2758
2019-09-17,85265,913,50.00,2765
2019-09-18,85265,919,50.00,2758
2019-09-19,85265,917,50.00,2739
2019-09-20,85265,931,50.00,2696
2019-09-21,85265,937,50.00,2679
2019-09-22,85265,917,50.00,2784
